### 1. Sekcja odczytywania danych, określania typów danych cech.

In [144]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score


def ReadDataFrame(filename:str, sep:str =';', dec_sep:str =',') -> tuple[pd.DataFrame, int]:
    """Odczytuje plik o nazwie filename i wprowadza dane z pliku do ramki danych."""
    global dtypes
   
    Dataset = pd.read_csv(filename,
                        sep=sep, dtype = dtypes, decimal = dec_sep) #Wczytaj plik z danymi.

    n_rows:int  = Dataset.shape[0] #Liczba wszystkich wierszy w ramce danych

    return Dataset, n_rows


### 2. Sekcja statystyki opisowej zmiennych kategorycznych

### Częstość występowania  unikatowych klas dla danej zmiennej.

In [145]:
def CreateFreqTable(Dataset:pd.DataFrame, CatFeature:str) ->  pd.DataFrame:
    """Funkcja dla każdej unikatowej klasy z cechy CatFeature wylicza liczbę jej wystapień.
    Wynikiem funkcji jest pandowska ramka danych, która zawiera trzy kolumny o nazwach odpowiednio: 'CatFeature', 'CatFeature_coded' oraz 'count'.
    Kolumna CatFeature_coded zawiera zakodowane nazwy klas w postaci liczb całkowitych. Taka forma ułatwia odczytywanie etykiet na wykresach.
    """

    FreqTable:pd.DataFrame = Dataset[CatFeature].value_counts(sort = True,) #Stwórz proste podsumowanie częstotliwości występowania klas.

    return  FreqTable

def PlotBarPlot(FreqTable:pd.Series, CatFeature:str, Showxlabels:bool = False) -> None:
    """Funkcja rysuje histogram na bazie tabelki histogramowej. 
    1) FreqTable  - Tabela częstotliwości kategori danej zmiennej kategorycznej
    2) CatFeature  - Cecha kategoryczna, której histogram chcemy narysować.
    3) Showxlabel - Zmienna typu bool. Jeżeli ustawiona na True, to etykietki osi Ox są wyświetlane."""
    
    plt.figure(figsize = (10,5)) #Stwórz płótno, na którym  będzie rysowany wykres
 
    axes = sns.barplot(x = FreqTable.index, y = FreqTable.values)


    axes.set_ylabel(f"Częstość klasy") #Ustaw etykietke pionowej osi.

    axes.set_xticklabels([]) #Usuń etykiety tyknięć na osi Ox.
    axes.spines[["top", "right"]].set_visible(False)

    axes.set_title(f"Histogram klas cechy {CatFeature}") #Ustaw tytuł wykresu.

    axes.set_ylim(0, 1.05*np.max(FreqTable))

    if Showxlabels == True:
        axes.set_xticklabels(labels = FreqTable.index)
        


def AggregateRarestClasses(Dataset:pd.DataFrame, CatFeature:str, Histogram:pd.Series, q_degree:float = 0.15) -> None:
    """Klasy, które występują rzadziej niż wartość wskaźnika threshold_frequent, dostaną etykietę "Other". Etykietki pozostałych klas nie ulegną zmian.
    1) Dataset - oryginalny zestaw danych.
    2) CatFeature - Zmienna kategoryczna, której rzadkie klasy chcemy zagregować.
    3) Histogram - Histogram cechy CatFeature
    4) q_degree  - stopień kwantylu. 
    Funkcja jedynie modyfikuje zestaw danych, dlatego właśnie nie zwraca żadnej wartości."""
    frequent_threshold:float = Histogram.quantile(q = q_degree)

    Dataset[CatFeature] = Dataset[CatFeature].apply(func = lambda categ: "Other" if Histogram[categ] < frequent_threshold else categ)

In [146]:
def DeleteFutileColsAndObs(Dataset:pd.DataFrame) -> pd.DataFrame:
    """Funkcja usuwa quasi-identyfikator zmienną oraz jedną obserwację, która zawiera klasę, która występuje tylko raz."""

    Dataset.drop(columns = ["Model"], inplace = True)

    Dataset = Dataset.query('`Fuel Type` != "N"')

    return Dataset
    

### 3. Cechy numeryczne ciągłe.

### Badanie zależności poziomu emisji dwutlenku węgla w zależności od wielkości spalania paliwa na autostradzie i w mieście dla różnych typów paliwa.
### Dodatkowo zbadamy współczynniki korelacji między tymi cechami ciągłymi.



In [147]:
def ComputeAndDrawCorrelationMatrix(Dataset:pd.DataFrame, FloatFeatures: list[str]) -> None:
    """Funkcja wylicza macierz korelaji dla zmiennych z listy FloatFeatures. Ponadto, rysuje tę macierz korelacji na wykresie, aby można
    było sobie uzmysłowić relacje między zmiennymi
    1) Dataset - oryginalny zbiór danych
    2) FloatFeatures  - zmienne ciągłe"""
    CorrMatrix:pd.DataFrame =  Dataset[FloatFeatures].corr(method = "pearson")

    plt.figure(figsize=(8, 6))

    sns.heatmap(CorrMatrix, annot=True, cmap='magma', vmin=-1, vmax=1)
    plt.title('Macierz korelacji dla zmiennych ciągłych')
    plt.show()



In [148]:
def Discretize(Dataset:pd.DataFrame, target_var:str, target_var_discr:str, bins:list[float]) -> None:
    """Funkcja dokonuje dyskretyzacji (interwalizacji) zmiennej docelowej ciągłej.
    1) Dataset - oryginalny zbiór danych.
    2) target_var - zmienna docelowa
    3) bins - kubełki, które określają granicę  interwałów zmiennej docelowej."""
    labels = [i for i in range(len(bins)+1)]
    bins = [-float("inf")] + bins + [float('inf')]


    discretized_feature = pd.cut(x = Dataset[target_var], 
                                 bins = bins, 
                                 labels = labels)

    Dataset[target_var_discr] = discretized_feature


In [149]:
def NarysujGęstości(Dataset:pd.DataFrame, FloatFeatures:list[str], Condition:str | None = None) -> None:
    """Ta funkcja rysuje wykresy gęstości prawdopodobieństwa dla zmiennych ciągłych.
    1) Dataset - oryginalny zestaw danych.
    2) FloatFeatures - zmienne numeryczne"""

    for floatFeature in FloatFeatures:
        figure = plt.figure(num = f"KDE_plot_{floatFeature}")
        axes = figure.add_subplot()

        sns.kdeplot(data = Dataset, x = floatFeature, ax = axes, hue = Condition)
        axes.set_title(f"Wykres gęstości prawdopodobieństwa dla zmiennej {floatFeature}")


def NarysujPudełko(Dataset:pd.DataFrame, FloatFeatures:list[str], Condition: str | None = None) -> None:
    """Ta funkcja rysuje wykresy pudełkowe dla zmiennych ciągłych.
    1) Dataset - oryginalny zestaw danych.
    2) FloatFeatures - zmienne numeryczne"""

    for floatFeature in FloatFeatures:
        figure = plt.figure(num = f"BOX_plot_{floatFeature}")
        axes = figure.add_subplot()

        sns.boxplot(Dataset, x = floatFeature, hue = Condition)

        axes.set_title(f"Wykres pudełkowy dla zmiennej {floatFeature}")


def NarysujSkrzypce(Dataset:pd.DataFrame, FloatFeatures:list[str], Condition:str | None = None) -> None:
     """Ta funkcja rysuje wykresy skrzypcowe dla zmiennych ciągłych.
    1) Dataset - oryginalny zestaw danych.
    2) FloatFeatures - zmienne numeryczne
    3) Condition = Pewna zmienna kategoryczna, za pomocą której stworzą się warunkowe wykresy skrzypcowe ze względu przynależność do klasy."""

     for floatFeature in FloatFeatures:
        figure = plt.figure(num = f"VIOLIN_plot_{floatFeature}")
        axes = figure.add_subplot()

        sns.violinplot(Dataset, x = floatFeature, hue = Condition)

        axes.set_title(f"Wykres skrzypcowy dla zmiennej {floatFeature}")

        axes.legend([])

        axes.grid(True, alpha = 0.6)
        axes.spines[['top','right']].set_visible(False)



def NarysujWykresParowy(Dataset:pd.DataFrame,  Condition: str | None = None) -> None:
    """Funkcja rysuje wykres parowy dla wszystkich par zmiennych ciągłych.
    """
    sns.pairplot(Dataset, hue = Condition,diag_kind = "kde")



In [150]:
# def ZakodujDane(Dataset:pd.DataFrame, CatFeatures:list[str], NumFeatures:list[str]) -> pd.DataFrame:
#     """Funkcja koduje  zmienne niezależne kategoryczne za pomocą transformatora OHE."""
#     cat_col_transformer = ColumnTransformer(transformers = 
#                                             [('OHE',OneHotEncoder(sparse_output = False), CatFeatures)],
#                                             remainder = "passthrough",)
    
#     #Przekształć zmienną X za pomocą określonego wyżej transformatora. 
#     X_coded = cat_col_transformer.fit_transform(X = Dataset)
    
#     CatFeatures_coded = cat_col_transformer.named_transformers_["OHE"].get_feature_names_out(input_features = CatFeatures,)

#     Dataset_coded = pd.DataFrame(data = X_coded, columns =  np.concatenate((CatFeatures_coded, NumFeatures)))
    
#     return Dataset_coded



# def TransformujDane(X: pd.DataFrame,  numpredictors:list[str], pca_predictors:list[str]) -> tuple[ np.ndarray, ColumnTransformer]:
#     """Funkcja stosuje standaryzacje dla tablicy, która zawiera już zmienne będące liczbami.
#     Numpredictors to zmienne, które zostaną poddane procesowi standaryzacji. pca_predictors to cechy, dla których składowe główne zostaną wyliczone"""
#     col_transformer = ColumnTransformer(transformers = [("Scaler", StandardScaler(), numpredictors) ,
#                                                              ('PCA', PCA(n_components = 1), pca_predictors)], remainder = "passthrough")


#     return col_transformer.fit_transform(X = X),  col_transformer



In [151]:
from sklearn.linear_model import LogisticRegression

# Definicja modeli z hiperparametrami
Models = {
    "DrzewkoDecyzyjne": DecisionTreeClassifier(criterion="gini", splitter="best", min_samples_split=10), 
    "LasLosowy": RandomForestClassifier(n_estimators=15, criterion='gini'), 
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "RegresjaLiniowa": MultiOutputRegressor(estimator = LinearRegression()),
    "RegresjaLogistyczna": LogisticRegression()
}                

# #Słownik do przechowywania hiperparametrów modeli:
# Models_hipparams = {"DrzewkoDecyzyjne":{"criterion":['gini','entropy'],
#                                         "splitter":['best','random'],
#                                         "min_samples_split":[2,3,4],
#                                         "min_samples_leaf":[2,3,4]},

#                     "LasLosowy":{"n_estimators":list(range(5, 50, 5)),
#                                   "criterion":['gini','entropy'],
#                                         "min_samples_split":[3,3,4],
#                                          "min_samples_leaf":[2,3,4]},
                                        
#                     "KNN": {"n_neighbors":list(range(1, 10, 1)),
#                             "p":[1,2]},
                            
#                         "RegresjaLogistyczna": {
#                             "penalty":['l1','l2'],
#                              "solver":['lbfgs','liblinear','newton-cg',],
#                              'multi_class':['auto','ovr','multinomial']
#                         }
#                             }


#Słownik do przechowywania hiperparametrów modeli:
Models_hipparams = {"DrzewkoDecyzyjne":{"criterion":['gini'],
                                        "splitter":['best'],
                                        "min_samples_split":[2],
                                        "min_samples_leaf":[2]},

                    "LasLosowy":{"n_estimators":list(range(5, 11, 5)),
                                  "criterion":['gini','entropy'],
                                        "min_samples_split":[2],
                                         "min_samples_leaf":[2]},
                                        
                    "KNN": {"n_neighbors":list(range(1, 3, 1)),
                            "p":[1]},
                            
                        "RegresjaLogistyczna": {
                            "penalty":['l1','l2'],
                             "solver":['lbfgs'],
                             'multi_class':['auto']
                        }, "RegresjaLiniowa":{}
                            }



def TrainAndTestModelStat(Model, X_train:pd.DataFrame, y_train:pd.DataFrame, X_test:pd.DataFrame) -> np.ndarray:
    """Funkcja trenuje model za pomocą danyc treningowych (X_train, y_train), a następnie dokonuje predyckcji etykiet klasy docelowej i zwraca przewidywane etykietki.
    Jeżeli model jest regresją liniową, przewidziane etykietki są tablicą wymiaru (n_test, n_outputs), gdzie n_outputs to liczba klas emisji.
    W przeciwnym wypadku  etykietki są wymiaru (n_test, 1)"""
    Model.fit(X = X_train, #Znajdź optymalne parametry dla danego Modelu, eksponując model na dane treningowe.
              y = y_train) \

    return Model.predict(X_test) # Dokonaj przewidywań etykietki klas. Nie zwracamy od razu metryki dokładności. Przewidziane etykietki pomogą nam metrykę policzyć oraz ułatwią
                                 #stworzenie macierzy pomyłek. \



def ComputeAccuracy(TruevsPrediction:pd.DataFrame, n_splits:int = 5) -> pd.DataFrame:
    """Funkcja, na podstawie tabeli porównań etykiet prawdziwych i przewidywanych, oblicza pewną miarę dokładności (accuracy_score) dla wszystkich modeli i dla wszystkich  powtórzeń."""

    MetricComparison = np.zeros(shape = [n_splits, len(Models.keys())], 
                                 dtype = np.float64)
    
    
    for model_indx, model in enumerate(Models.keys()):
        for i in range(n_splits):
            y_true = TruevsPrediction[(model, i, "True")]
            y_pred = TruevsPrediction[(model, i, "Pred")]

            perfomance_metric = accuracy_score(y_true = y_true, 
                                               y_pred = y_pred)

            MetricComparison[i, model_indx] = perfomance_metric

    return pd.DataFrame(data = MetricComparison, 
                                     columns = list(Models.keys()))



def ComputeMetric(TruevsPrediction:pd.DataFrame, metric) -> pd.DataFrame:
    """Funkcja wylicza precyzje albo wrażliwość albo f1"""
    MetricComparison = np.zeros(shape = [n_splits, len(Models.keys())], 
                                 dtype = np.float64)
    
    for model_indx, model in enumerate(Models.keys()):
        for i in range(n_splits):
            y_true = TruevsPrediction[(model, i, "True")]
            y_pred = TruevsPrediction[(model, i, "Pred")]

            perfomance_metric =  metric(y_true =y_true,
                                                  y_pred = y_pred, 
                                                  average = "weighted",)

            MetricComparison[i, model_indx] = perfomance_metric


    return pd.DataFrame(data = MetricComparison, 
                                     columns = list(Models.keys()))



In [152]:
def PlotlineScores(perfomance_df: pd.DataFrame, metric_name:str, n_splits:int, params_type:str = "bez strojenia") -> None:
    """Stwórz wykresy liniowe obrazujące dynamikę zmian dokładności modeli."""
    figure = plt.figure()
    axes = figure.add_subplot()
    
    x_values:list[int] = list(range(0,n_splits))

   
    for model_name in Models.keys():
        axes.plot(x_values, 
                  perfomance_df[model_name])
    
    #Tablica, która przechowuje mediane wartości metryki dokładności w i-tym podziale.
    averages:list[float] = []
    
    for i in range(n_splits):
        averages.append(perfomance_df.iloc[i, :].median())
    
    axes.plot(x_values, averages, linestyle = "dashed", linewidth = 4)

    axes.legend(list(Models.keys()) +["MetricMedian"])
    axes.set_title(f"Dynamika zmian metryki {metric_name} dla modeli, wersja {params_type}")

    axes.set_xlabel("Numer iteracji") #Ustaw ładną etykietkę osi Ox
    axes.set_ylabel(f"{metric_name}") #Ustaw ładną etykietkę osi Oy

    axes.set_xticks(x_values) #Ustaw wartości tyknięć na osi Ox
    
    axes.grid(True) #Dodaj linie siatki


    axes.spines[['top','right']].set_visible(False) #Usuń kreskę górną oraz dolną.






def PlotConfussionMatrices(model_name:str, y_true:np.ndarray, y_predicted: np.ndarray, type:str = "bez strojenia"):
    """Funkcja ta rysuje macierz pomyłek dla  danego modelu. 
        1) y_true  są etykietkami rzeczywistymi
        2) y_predicted są etykietkami przewidzianymi
        3) type jest to typ strojenia parametrów. Może być ten tryb statyczny (wtedy piszemy type = "stat") albo dynamiczny (wtedy piszmey type = "dyna")
        """
    axes = plt.figure(num = f"{model_name}_conf_matrix_{type}").add_subplot()


    cos = ConfusionMatrixDisplay.from_predictions(y_true = y_true, y_pred = y_predicted, ax = axes, normalize = "true")
    axes.set_title(f"ConfMatrix dla  modelu {model_name}, wersja {type}")










def MacierzPomylek(TrueVSPrediction:pd.DataFrame, n_splits:int, type:str = "bez strojenia"):
    """Funkcja wylicza macierze pomyłek dla wszystkich modeli, a następnie obrazuje te macierze na wykresie.
    1) TrueVSPrediction: macierz zawierająca etykietki przewidywane oraz rzeczywiste
    2) Liczba podziałów zbioru dataset na zbiór treningowy i testowy.
    3) type = rodzaj strojenia parametrów (statyczny albo dynamiczny)"""

    for model_name in Models.keys():
        y_true = TrueVSPrediction[(model_name, 0, "True")] #Znajdź kolumnę prawdziwcych etykiet.
        y_pred = TrueVSPrediction[(model_name, 0, "Pred")] #Znajdź kolumne przewidywanych etykiet.
        
        PlotConfussionMatrices(model_name, y_true, y_pred, type) #Narysuj macierz pomyłek.







def StworzTabelkePorównawczą() -> pd.DataFrame:
    global Models, n_splits
    """Funkcja stwarza tabelkę porównawcza, która jest po prostu ramką pandas. Indeksy kolumn są trzypoziomowe. Na najwyższym poziomie jest nazwa modelu, niżej jest
    numer podziału, a na końcu tyb etykiet (prawdziwe etykiety lub prawdziwe etykiety)"""


    Indeces = pd.MultiIndex.from_product( [list(Models.keys()), range(n_splits), ["True", "Pred"] ] #Stwórz  hierarchiczny system indeksów dla kolumn.
                                        ,names = ["Model", "iter_no", "array_type"]) #Nadaj poszczególnym poziomom wyjaśnialne i sensowne nazwy.

    return  pd.DataFrame(data = None, 
                                    columns = Indeces)

def PorównajZIBeZStrojeniaModel(Metric_comparison_stat:pd.DataFrame, Metric_comparison_dyna:pd.DataFrame, metric_name:str,
                           n_splits:int) -> None:
    """Funkcja porónuje wartości metryk dokładności dla stojonych i niestrojonych modeli indywidualnie."""
    global Models

    models_name: list[str] = list(Models.keys()) #Znajdź listę nazw modeli.

    x_values:list[int] = list(range(1, n_splits+1))

    for model_name in models_name:
        #Stwórz okienko, na którym będzie wyświetlane porównanie między niedostrojonym, a dostrojonym modelem.
        okno: plt.figure = plt.figure( num = f"{model_name} (un)tuned metric comparison for {metric_name}")
        #Stwórz osie.
        osie = okno.add_subplot()

    
        M_untuned:pd.Series = Metric_comparison_stat[model_name] 
        M_tuned: pd.Series = Metric_comparison_dyna[model_name]
     
        osie.plot(x_values,  M_untuned)
        osie.plot(x_values, M_tuned)

        osie.legend(["Niedostrojony",'Dostrojony'])


        osie.grid(True)
        osie.spines[['top','right']].set_visible(False)
        osie.set_title(f"Porównanie niestrojonego i strojonego {model_name}, metryka {metric_name}")

        osie.set_xlabel("Numer powtórzenia")
        osie.set_ylabel(f"Wartość metryki {metric_name}")
        osie.set_xticks(x_values)





### Przygotowanie metody porównaczej.


In [153]:
from sklearn.feature_selection import SequentialFeatureSelector as SFS
from sklearn.pipeline import Pipeline

class TrainTestModels():
    def __init__(self,Dataset:pd.DataFrame, target_var_disc:str, Cat_features:list[str], Num_features:list[str], Models:dict[str, "estimator"], Models_hipparams:dict[str, dict],
                 n_splits:int = 5, train_size:float = 0.8, test_size:float = 0.2, Cat_predictors:list[str]  = [], Num_predictors:list[str] = [], PCA_predictors:list[str]= []) -> None:
        """Konstruktor klasy, która trenuje modele. 
        Opis argumentów konstruktora:
        ---------
        Dataset: pd.DataFrame - Ramka danych typu pandas, która przechowuje zmienne objaśniające i zmienną objaśnianą. \n
        target_var_disc:str - Nazwa zmiennej objaśnianej. \n
        Cat_features:list[str] - Lista zmiennych kategorycznych. \n
        Num_features:list[str] - Lista zmiennych numerycznych. \n
        Cat_predictors:list[str] - Lista predyktorów kategorycznych. Predyktory kategoryczne nie zostały jeszcze zakodowane. \n
        Num_predictors:list[str] - Lista predyktorów numerycznych. \n
        PCA_predictors:list[str] - Lista predyktorów, które mają ulec analizie składowych głównych.
        Models:dict[str, "Estimator"] - Słownik, którego wartościami są instancje modeli, które chcemy wytrenować. Kluczami są umowne nazwy modeli. \n
        Models_hipparams:dict[str, dict] - Słownik, którego kluczami są umowne nazwy modeli, a którego wartościami są siatki parametrów danego modelu. \n
        n_splits:int - Liczba podziałów na zbiór treningowy i testowy. \n
        train_size:float - Odsetek obserwacji treningowych. \n

        test_size:float - Odsetek obserwacji testowych. \n
        ---------
        """
        self.Dataset:pd.DataFrame = Dataset 
        
        self.Cat_features = Cat_features
        self.Num_features = Num_predictors

        self.target_var_discr:str = target_var_disc
        self.Cat_predictors:list[str] = Cat_predictors
        self.Num_predictors:list[str] = Num_predictors
        self.PCA_predictors:list[str] = PCA_predictors

        self.Models:dict[str, "estimator"] = Models
        self.Models_hipparams:dict[str, dict] = Models_hipparams

        self.n_splits:int = n_splits
        self.train_size:float = train_size
        self.test_size:float = test_size


        self.PrzygotujRamkiPorownawcze()
        


    @staticmethod
    def StworzRamkePorownawcza(Models: dict[str, "estimator"], n_splits:int = 5) -> pd.DataFrame:
        """Funkcja stwarza tabelkę porównawcza, która jest po prostu ramką pandas. Indeksy kolumn są trzypoziomowe. Na najwyższym poziomie jest nazwa modelu, niżej jest
        numer podziału, a na końcu tyb etykiet (prawdziwe etykiety lub przewidywane etykiety).  \n 
        Opis argumentów: 
        ---------
        Models:dict[str, "estimator"] - Słownik, którego wartościami są instancje modeli, które trenujemy, a którego kluczami są umowne nazwy modeli. \n
        n_slits:int - Liczba podziałów zbioru na zbiór uczący i zbiór testowy. \n.
        ---------
        """

        Indeces = pd.MultiIndex.from_product( [list(Models.keys()), range(n_splits), ["True", "Pred"] ] #Stwórz  hierarchiczny system indeksów dla kolumn.
                                            ,names = ["Model", "iter_no", "array_type"]) #Nadaj poszczególnym poziomom wyjaśnialne i sensowne nazwy.

        return  pd.DataFrame(data = None,  columns = Indeces)


    def PrzygotujRamkiPorownawcze(self,) -> None:
        """Ta metoda dla każdego rodzaju modeli (niestrojony, strojony, strojony+fs) tworzy ramki porównawcze."""

        self.FactVsPrediction_untuned:pd.DataFrame =  TrainTestModels.StworzRamkePorownawcza(Models = self.Models, n_splits = n_splits)
        self.FactVsPrediction_tuned:pd.DataFrame = TrainTestModels.StworzRamkePorownawcza(Models = self.Models, n_splits = n_splits)
        self.FactVsPrediction_FS:pd.DataFrame = TrainTestModels.StworzRamkePorownawcza(Models = self.Models, n_splits = n_splits)



    def ZakodujZmienneKategoryczne(self , X:pd.DataFrame, Cat_predictors:list[str], Num_predictors:list[str]) -> pd.DataFrame:
        """Funkcja koduje  zmienne niezależne kategoryczne za pomocą transformatora OHE.
        Opis argumentów:
        ---------
        X:pd.DataFrame - Zbiór danych z predyktorami. \n
        Cat_predictors:list[str] - Lista predyktorów kategorycznych. \n
        Nun_predictors:list[str] - Lista predyktoró numerycznych. \n


        Co funkcja zwraca:
        Zbiór danych, którego predyktory kategoryczne zostały zakodowane metodą OHE.
        """
        cat_col_transformer = ColumnTransformer(transformers = 
                                                [('OHE',  OneHotEncoder(sparse_output = False), Cat_predictors)],
                                                remainder = "passthrough",)
        

        #Przekształć zmienną X za pomocą określonego wyżej transformatora. 
        X_coded:np.ndarray = cat_col_transformer.fit_transform(X = X)

        #Znajdź nowe nazwy zmiennych kategorycznych.
        CatFeatures_coded:list[str] = cat_col_transformer.named_transformers_["OHE"].get_feature_names_out(input_features = Cat_predictors,) 

        return pd.DataFrame(data = X_coded, columns =  np.concatenate((CatFeatures_coded, Num_predictors)))
         
    
    
    def TransformujZmienneNumeryczne(self, X: pd.DataFrame,  Num_predictors:list[str], PCA_predictors:list[str]) -> tuple[ np.ndarray, ColumnTransformer]:
        """Funkcja aplikuje transformacje na predyktorach numerycznych (skalowanie standardowe, analiza składowych głównych)
        Opis argumentów:
        X:pd.DataFrame - Treningowy zbiór danych z predyktorami. \n
        Nun_predictors:list[str] - Lista predyktoró numerycznych. \n
        PCA_predictors:list[str] - Lista predyktorów dla których ma zostać wykonana analiza składowych głównych.

        Co funkcja zwraca:
        Przekształcony zbiór predyktorów oraz wytrenowany transformator.

        """

        No_PCA_num_predictors:list[str] = [var for var in Num_predictors if var not in PCA_predictors] #Znajdź zmienne numeryczne, których nie poddajemy procesowi analizy składowych głównych.

        Num_transformer:Pipeline = Pipeline(steps = [("Scalling", StandardScaler())]) #Rurociąg dla niePCA predyktorów
        PCA_transformer: Pipeline = Pipeline( steps = [ ("Scalling", StandardScaler(), ),  #Rurociąg dla predyktorów PCA
                                                       ("PCA",  PCA(n_components = 0.9))     ])

        #Zdefiniuj rownoległy transformator dla kolumn PCA  i kolumn niePCA
        col_transformer: ColumnTransformer = ColumnTransformer(transformers = 
                                                               [("Num_trans", Num_transformer, No_PCA_num_predictors),
                                                                               ("PCA", PCA_transformer, PCA_predictors)],
                                                                               remainder = "passthrough")
                                 
        #Wytrenuj kolumnowy_transformator                                                                
        col_transformer.fit(X = X)

        X_transformed:np.ndarray = col_transformer.transform(X = X)
        
        return X_transformed,  col_transformer


    


    def TrenujTestujManualnie(self, X:pd.DataFrame, y:pd.Series, train_indx:np.ndarray, test_indx:np.ndarray) -> None:
        """Ta metoda trenuje i testuje modele uczenia maszynowego za pomocą ręcznie dobranych predyktorów.
        Opis argumentów:
        X:pd.DataFrame - Zbiór predyktorów
        y:pd.Series - Zmienna docelowa
        train_indx:np.ndarray - Zbiór indeksów treningowych
        test_indx: np.ndarray - Zbiór indeksów testowychc.
        """
        X_encoded:pd.DataFrame = self.ZakodujZmienneKategoryczne(X = X, 
                                           Cat_predictors = self.Cat_predictors,
                                            Num_predictors = self.Num_predictors)
        

        for model_name in self.Models.keys():
            model: 'estimator' = self.Models[model_name] #Instancja danego modelu.

            
            model_hipparams: dict[str, dict] = self.Models_hipparams[model_name] #Siatka hiperparametrów modelu.

            if model_name != "RegresjaLiniowa":
                X_train:pd.DataFrame = X_encoded.iloc[train_indx, :] #Treningowy zbiór predyktorów, którego kategoryczne zmienne zostały zakodowane.
                X_test:pd.DataFrame = X_encoded.iloc[test_indx, :] #Testowy zbiór predyktorów, którego kategoryczne zmienne zostały zakodowane.
        

                y_train:pd.Series = y[train_indx] #Testowy zbiór zmiennej docelowej.   
                y_test:pd.Series = y[test_indx] #Testowy zbiór zmiennej docelowej.
                print(X_train.shape, "Próba przed \n")

                #Zdobądź przekształcony zbiór treningowy oraz transformator kolumn, który został na nim wytrenowany.
                X_train, Transformator = self.TransformujZmienneNumeryczne(X = X_train, 
                                                                           Num_predictors = self.Num_predictors, 
                                                                           PCA_predictors = self.PCA_predictors)
                
                X_test = Transformator.transform(X = X_test)



                

    def PodzielZbiorDanych(self):
        """"Funkcja wydobywa indeksy treningowe i indeksy testowe w n_splits podziałach, które następnie przekazuje do metody TrenujTestujManualnie"""
        #Zdefiniuj stratyfikowany podział szufladkowy.
        SSS_inst: StratifiedShuffleSplit = StratifiedShuffleSplit(n_splits = self.n_splits, 
                                                                  test_size = self.test_size, 
                                                                  train_size = self.train_size)
        
        self.X:pd.DataFrame = self.Dataset[ self.Cat_predictors + self.Num_predictors] #Ramka danych zawierająca predyktory, które nie zostały poddane obróbce wstępnej.
        self.y:pd.DataFrame = self.Dataset[self.target_var_discr] #Zdyskretyzowana zmienna docelowa.


        iter_indx:int  #Zmienna, która wskazuje na numer powtórzenia pętli.
        indx_tup: tuple[np.ndarray, np.ndarray] #Dwuelementowa krotka, która zawiera indeksy treningowe oraz indeksy testowe.

        for iter_indx, indx_tup in enumerate(SSS_inst.split(X = self.X, y = self.y)):
            train_indx:np.ndarray #Indeksy treningowe. 
            test_indx:np.ndarray  #Indeksy testowe.

            train_indx, test_indx = indx_tup

            self.TrenujTestujManualnie(X = self.X, y = self.y, 
                                       train_indx = train_indx, test_indx = test_indx)

        pass


        

def TrenujTestujModele(X:pd.DataFrame, y:pd.DataFrame, y_OHE:np.ndarray, cat_predictors:list[str], num_predictors:list[str],
                       Models:dict[str, "estimator"], Models_hipparams:dict[str,  dict ], n_splits:int = 5, train_size:float = 0.8, test_size:float = 0.2) -> tuple[pd.DataFrame, pd.DataFrame]:
    

    
    TrueVSPrediction_stat:pd.DataFrame = StworzTabelkePorównawczą() #Wersja niestrojona
    TrueVSPrediction_dyna:pd.DataFrame = StworzTabelkePorównawczą() #Wersja strojona
                                                        #Wersja featureselection + tuning
    
    TrueVSPrediction_fs: pd.DataFrame = StworzTabelkePorównawczą()

    SSS = StratifiedShuffleSplit(n_splits = n_splits, train_size = train_size, test_size  = test_size)
   

    for iter_indx, indx in enumerate(SSS.split(X = X, y = y )): #Wydobadź indeksy treningowe i testowe.
        train_indx, test_indx = indx #Znajdź indeksy treningowe oraz indeksy testowe.
    
        X_train =  X.iloc[train_indx, ]
        X_test = X.iloc[test_indx, ]
    
    
        X_train ,Scaler = TransformujDane(X_train, numpredictors = num_predictors, pca_predictors= ["Fuel Consumption City (L/100 km)", "Fuel Consumption Hwy (L/100 km)",
                                                                                   "Fuel Consumption Comb (L/100 km)","Fuel Consumption Comb (mpg)"])
        X_test = Scaler.transform(X_test) #Dokonaj skalowania na zbiorze testowym za pomocą parametrów wydobytych ze zbioru treningowego.

    
        for model_name in Models.keys():
            model:'Estimator' = Models[model_name] #Zrekrutuj model.
            model_hiperparams = Models_hipparams[model_name] #Wydobądź siatek hiperparametrów do przeszukania.
            
            if model_name != "RegresjaLiniowa":
                y_train = np.array(y.iloc[train_indx, :]).ravel()
                y_test =  np.array(y.iloc[test_indx, :]).ravel()

                #Statyczne dobieranie hiperparametrów.
                y_pred_stat: np.ndarray = TrainAndTestModelStat(model, X_train, y_train, X_test) #Etykietki przewidziane.


                #Miejsce na dynamiczne strojenie parametrów.
                GridSearch = GridSearchCV(estimator = model, param_grid = model_hiperparams)
                fitted_model = GridSearch.fit(X = X_train, y = y_train) #To jest instancja modelu z najbardziej optymalnymi hiperparametrami.

                #Znajdź predyktowane etykietki
                y_pred_dyna:np.ndarray = fitted_model.predict(X_test)


                #Doklej etykietki przewidziane do ramki danych porównującej statycznie-strojone modele.
                TrueVSPrediction_stat[(model_name, iter_indx, "True")] = y_test
                TrueVSPrediction_stat[(model_name, iter_indx, "Pred")] = y_pred_stat


                #Doklej etykietki przewidziane do ramki danych porównującej dynamicznie-strojone modele.
                TrueVSPrediction_dyna[(model_name, iter_indx, "True")] = y_test
                TrueVSPrediction_dyna[(model_name, iter_indx, "Pred")] = y_pred_dyna


            else:
                #Regresja liniowa niestety ma wielowymiarowe-wyjście.
                y_train = y_OHE[train_indx] #Weź etykietki docelowe treningowe.
                y_test = y_OHE[test_indx].argmax(axis = 1) #Weź etykietki docelowe testowe.

                #Znajdź predyktowane etykietki
                y_pred: np.ndarray = TrainAndTestModelStat(model, X_train, y_train, X_test).argmax(axis = 1) 
                

                #Doklej etykietki przewidziane do ramki danych porównującej statycznie-strojone modele.
                TrueVSPrediction_stat[(model_name, iter_indx, "True")] = y_test
                TrueVSPrediction_stat[(model_name, iter_indx, "Pred")] = y_pred


                #Doklej etykietki przewidziane do ramki danych porównującej dynamicznie-strojone modele.
                TrueVSPrediction_dyna[(model_name, iter_indx, "True")] = y_test
                TrueVSPrediction_dyna[(model_name, iter_indx, "Pred")] = y_pred

            print(f"{model_name} {iter_indx}")

         
            #Miejsce na strojenie modeli wraz z wyborem cech
            SFS_inst = SFS(estimator = model, n_features_to_select = X.shape[1]//10, direction = "forward")

            Rura: Pipeline = Pipeline(steps = [      ("SFS", SFS_inst ),    (model_name, model)      ])
                                                                    #Wyciągasz wartości parametru "param" dla modelu "model_name"
            model_hiperparams_fs = { f"{model_name}__{param}": model_hiperparams[param] for param in model_hiperparams.keys() }

            GridSearch_FS = GridSearchCV(estimator = Rura, param_grid = model_hiperparams_fs)

            fitted_model_fs =  GridSearch_FS.fit(X = X_train, y = y_train)

            #Znajdź predyktowane etykietki
            y_pred_fs:np.ndarray = fitted_model_fs.predict(X_test)

            TrueVSPrediction_fs[(model_name, iter_indx, "True")] = y_test
            TrueVSPrediction_fs[(model_name, iter_indx, "Pred")] = y_pred_fs


    return TrueVSPrediction_stat, TrueVSPrediction_dyna, TrueVSPrediction_fs




In [154]:
def PorównajModele(TrueVSPrediction_df_stat:pd.DataFrame,
                   TrueVSPrediction_df_dyna:pd.DataFrame, 
                   n_splits:int ) -> None:
    """Funkcja wylicza miary dokładności dla wszystkich modeli. 
    Pierwsze dwa argumenty są ramkami danych, których wartościami są prawdziwe lub przewidziane etykietki. 
    Każda z tych ramek ma wielopoziomy system indeksowania kolumn, który jest postaci: (model_name, iter_indx, type).
    model_name jest nazwą danego modelu. iter_indx wskazuje na to, w którym podziale etykietki są przewidziane. type = "True" albo type = "Pred". 
    Gdy type = "True", etykietki są rzeczywiste, w przeciwnym wypadku są przewidziane."""


    Miary = {"Accuracy" : ComputeAccuracy, "Precision":precision_score,
             "Recall":recall_score, "F1": f1_score}
    

    for miara in Miary.keys():

        if miara == "Accuracy":
            Metric_comparison_stat = Miary[miara](TrueVSPrediction_df_stat) #Wartości metryki dokładności, jeżeli strojenie nie miało miejsca.
            Metric_comparison_dyna = Miary[miara](TrueVSPrediction_df_dyna) #Wartości metryki dokładności, jeżeli było strojenie.


            PlotlineScores(Metric_comparison_stat, miara, n_splits, "bez strojenia")
            PlotlineScores(Metric_comparison_dyna, miara, n_splits, "ze strojeniem")


            
        else:
            Metric_comparison_stat =  ComputeMetric(TrueVSPrediction_df_stat, metric = Miary[miara])
            Metric_comparison_dyna =  ComputeMetric(TrueVSPrediction_df_dyna, metric = Miary[miara])

            PlotlineScores(Metric_comparison_stat, miara, n_splits, "bez strojenia")
            PlotlineScores(Metric_comparison_dyna, miara, n_splits, "ze strojeniem")


        #Na sam koniec, dla każdego modelu indywidualnie, porównaj jego wersję niedostrojoną i strojoną.
        PorównajZIBeZStrojeniaModel(Metric_comparison_stat, Metric_comparison_dyna, 
                                    metric_name = miara,
                                      n_splits = n_splits)
     


    MacierzPomylek(TrueVSPrediction_df_stat, n_splits) #Wylicz oraz narysuj macierz pomyłek dla statycznego strojenia hiperparametrów
    MacierzPomylek(TrueVSPrediction_df_stat, n_splits, type = "Ze strojeniem") #Wylicz oraz narysuj macierz pomyłek dla dynamicznego strojenia hiperparametrów.


In [155]:
def ManualEmissionAnalysis(Filename:str, seperator:str, dec_sep: str, dtypes: dict[str, str], bins:list[float], show_plots: bool = True) -> None:
    """Wielka analiza danych"""
    global target_var, n_splits, train_size, test_size, Models, Models_hipparams


    Dataset, n_rows = ReadDataFrame(Filename, seperator, dec_sep) #Odczytaj zestaw danych oraz liczbę wszystkich obserwacji.
    Dataset = Dataset.head(2500)
    
    Features = Dataset.columns #Znajdź listę wszystkich cech w zbiorze danych.

    CatFeatures: list[str] = [feature for feature in Features if dtypes[feature] == "category"] #Cechy kategoryczne.
    FloatFeatures: list[str] = [feature for feature in Features if dtypes[feature] is np.float64] #Cechy ciągłe.


    #RYSOWANIE HISTOGRAMÓW DLA ZMIENNYCH KATEGORYCZNYCH
    #Agregacja rządkich klas dla zmiennych kategorycznych.
    for CatFeature in CatFeatures:
        Histogram:pd.Series = CreateFreqTable(Dataset = Dataset,  #Znajdź histogram unikatowych klas dla zmiennej CatFeature.
                                    CatFeature = CatFeature) 
        
        AggregateRarestClasses(Dataset, CatFeature, Histogram) #Zagreguj najrzadzsze klasy.

        Histogram_agg:pd.Series = CreateFreqTable(Dataset = Dataset,  #Zagregowany histogram.
                                                  CatFeature = CatFeature)
        
        if show_plots is True:
            PlotBarPlot(Histogram_agg, CatFeature)

    
    #Kasowanie zbędnej obserwacji oraz quasi-id kolumny
    Dataset = DeleteFutileColsAndObs(Dataset)

    #Dyskretyzacja zmiennej docelowej
    target_var_discr = target_var +"_disc"

    Discretize(Dataset, target_var, target_var_discr = target_var_discr,
            bins = bins)


    
    if show_plots is True: #Narysuj wykresy charakteryzujące (relacje między zmiennymi) oraz (charakterystyki zmiennych) na żądanie.
        #RYSOWANIE MACIERZY KORELACJI DLA ZMIENNYCH NUMERYCZNYCH
        ComputeAndDrawCorrelationMatrix(Dataset, FloatFeatures)

        #RYSOWANIE WYKRESÓW SKRZYPCOWYCH DLA Z MIENNYCH NUMERYCZNYCH
        NarysujSkrzypce(Dataset, FloatFeatures)

        #Rysowanie wykresu parowego dla zmiennych numerycznych
        NarysujWykresParowy(Dataset)

      
    

        target_var_discr_histogram:pd.Series = CreateFreqTable(Dataset, target_var_discr) #Znajdź histogram tabelkowy dla zdyskretyzowanej zmiennej celu.
        PlotBarPlot(target_var_discr_histogram, target_var_discr, Showxlabels = True) #Narysuj wykres słupkowy częstotliwości dla zmiennej celu zdyskretyzowanej.


        NarysujGęstości(Dataset, FloatFeatures, target_var_discr) #Wykresy gęstości warunkowe
        NarysujPudełko(Dataset, FloatFeatures, target_var_discr) #Wykresy pudełkowe warunkowe


        NarysujWykresParowy(Dataset, target_var_discr) #Wykresy parowe warunkowe
        

    #Ustal ostateczny zbiór predyktorów.
    Predictors = ['Make', "Vehicle Class",'Engine Size(L)','Cylinders','Transmission','Fuel Type',"Fuel Consumption City (L/100 km)", "Fuel Consumption Hwy (L/100 km)",
                                                                                   "Fuel Consumption Comb (L/100 km)","Fuel Consumption Comb (mpg)"]

    #Podziel zbiór predyktorów na zmienne numeryczne oraz zmienne kategoryczne odpowiednio.
    num_predictors:list[str] = [feature for feature in Predictors if dtypes[feature] is np.float64]
    cat_predictors:list[str] = [feature for feature in Predictors if dtypes[feature] == "category"]


    WielkiEstimator = TrainTestModels(Dataset = Dataset, target_var_disc = target_var_discr, Cat_features = CatFeatures, Num_features = FloatFeatures, 
                                      Models = Models, Models_hipparams = Models_hipparams, n_splits = 5, train_size = 0.8, test_size = 0.2, Cat_predictors = cat_predictors, 
                                      Num_predictors = num_predictors, PCA_predictors = ["Fuel Consumption City (L/100 km)", "Fuel Consumption Hwy (L/100 km)", 
                                                                             "Fuel Consumption Comb (L/100 km)","Fuel Consumption Comb (mpg)"])
    
    WielkiEstimator.PodzielZbiorDanych()
    
    
    
 

    

    # X:pd.DataFrame = Dataset[cat_predictors + num_predictors] #Tablica predyktorów.
    # y:pd.DataFrame = Dataset[[target_var_discr]] #Zmienna docelowa.

    # X_coded:pd.DataFrame = ZakodujDane(X, cat_predictors, num_predictors) #Zakoduj zmienne kategoryczne predykcyjne.

     
    # y_OHE:np.ndarray = OneHotEncoder(sparse_output = False).fit_transform(X = y) #Zakoduj zmienną docelową jako zbiór k kolumn binarnych. 
    #                                                                     #Tak zakodowana etykietka przyda się w wielowyjściowej regresji.


    # SSS = StratifiedShuffleSplit(n_splits = n_splits,  #Zdefiniuj stratyfikowany podział szufladkowy, tak aby zachować rozkład częstości  klas w zmiennej docelowej.
    #                              train_size =train_size, 
    #                              test_size = test_size)
    

    # TrueVSPrediction_stat, TrueVSPrediction_dyna, TrueVSPrediction_fs = TrenujTestujModele(X = X_coded, y = y, y_OHE = y_OHE, cat_predictors = cat_predictors,
    #                                                                                        num_predictors = num_predictors, Models = Models, Models_hipparams = Models_hipparams, 
    #                                                                                        n_splits = n_splits, train_size  = train_size, test_size = test_size)


                                                                        
    #PorównajModele(TrueVSPrediction_stat, TrueVSPrediction_dyna, n_splits)


In [156]:
target_var: str = "CO2 Emissions(g/km)" #To jest nazwa zmiennej docelowej.
n_splits = 15
train_size = 0.75
test_size  = 1 - train_size

dtypes = { "Make": "category", #Określ typ danych każdej cechy w ramce danych, która zostanie zaraz odczytana.
            "Model":"category",
            "Vehicle Class":"category",
            "Engine Size(L)":np.float64,
            "Cylinders":"category",
            "Transmission":"category",
            "Fuel Type":"category",
            "Fuel Consumption City (L/100 km)":np.float64,
            "Fuel Consumption Hwy (L/100 km)":np.float64,
            "Fuel Consumption Comb (L/100 km)":np.float64,
            "Fuel Consumption Comb (mpg)":np.float64,
            "CO2 Emissions(g/km)":np.float64}


ManualEmissionAnalysis("CO2Emission.csv", ';', ',', dtypes = dtypes,  bins = [150, 200, 300], show_plots = False)


# Pytania badawcze przykładowe:
# 1) Jak liczba klas docelowych wpływa na skuteczność metod? Czy skuteczność modelu domyślnie maleje, jeżeli liczba klas docelowych wzrośnie?
# 2) Jak skuteczne są metody proste w porównaniu z metodami bardziej zaawansowanymi.
# 3) Jak istotne jest strojenie parametrów? Czy statyczne strojenie parametrów ulega dynamicznego strojeniu parametrów.
# 4) Jak istotny jest wybór optymalnych cech? Czy należy uwzględnić wszystkie względne? A może wystarczy tylko kilka cech?
# 5) Jak prezentuje się dokładnośc predykcji w stosunku do poszczególnych klas? Czy klasy rzadsze są łatwiej przewidywalne?
        



#Alternatywny dla PCA: MDS, 
#Dlaczego regresja liniowa jest nieefektywna? Masked class problem.
#Porównaj czas uczenia się metod, zarówno dla statycznego strojenia i optymalnego strojenia parametrów.
#Prz


(2000, 90) Próba przed 

(2000, 87) Próba po 

(2000, 90) Próba przed 

(2000, 87) Próba po 

(2000, 90) Próba przed 

(2000, 87) Próba po 

(2000, 90) Próba przed 

(2000, 87) Próba po 

(2000, 90) Próba przed 

(2000, 87) Próba po 

(2000, 90) Próba przed 

(2000, 87) Próba po 

(2000, 90) Próba przed 

(2000, 87) Próba po 

(2000, 90) Próba przed 

(2000, 87) Próba po 

(2000, 90) Próba przed 

(2000, 87) Próba po 

(2000, 90) Próba przed 

(2000, 87) Próba po 

(2000, 90) Próba przed 

(2000, 87) Próba po 

(2000, 90) Próba przed 

(2000, 87) Próba po 

(2000, 90) Próba przed 

(2000, 87) Próba po 

(2000, 90) Próba przed 

(2000, 87) Próba po 

(2000, 90) Próba przed 

(2000, 87) Próba po 

(2000, 90) Próba przed 

(2000, 87) Próba po 

(2000, 90) Próba przed 

(2000, 87) Próba po 

(2000, 90) Próba przed 

(2000, 87) Próba po 

(2000, 90) Próba przed 

(2000, 87) Próba po 

(2000, 90) Próba przed 

(2000, 87) Próba po 



Co dalej do roboty?:
2) Zamknięćie wszystkich funkcji w jedną, potężna funkcję.
4) Liczenie wskaźników dokładności, jeżeli selekcja cech była wybierana automatycznie.